In [18]:
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.ndimage
import cv2
from sklearn.utils import shuffle
from tensorflow.contrib.layers import flatten
%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow.contrib'

In [ ]:
training_file = "data/train.pickle"
validation_file= "data/valid.pickle"
testing_file = "data/test.pickle"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test))

print("X_train shape:", X_train[0].shape)
print("y_train shape:", y_train.shape)
print("X_valid shape:", X_valid[0].shape)
print("y_valid shape:", y_valid.shape)
print("X_test shape:", X_test[0].shape)
print("y_test shape:", y_test.shape)

In [ ]:
n_train = len(X_train)
n_valid = len(X_valid)
n_test = len(X_test)
image_shape = X_train[0].shape
n_classes = len(np.unique(y_train))

In [ ]:
print("Number of training examples = ", n_train)
print("Number of validation example = ", n_valid)
print("Number of testing samples = ", n_test)
print("Shape of first training image = ", image_shape[0], "w x", image_shape[1], "h x", image_shape[2], "d")
print("Number of classes = ", n_classes)

In [ ]:
fig, axs = plt.subplots(2,10, figsize=(15,6))
fig.subplots_adjust(hspace=.2, wspace=.1)
axs = axs.ravel()

for i in range(20):
    index = random.randint(0,len(X_train))
    image = X_train[index]
    axs[i].axis('off')
    axs[i].imshow(image)
    axs[i].set_title(y_train[index])

In [ ]:
a = y_train
b = y_valid
c = y_test
bins = n_classes

plt.hist(a, bins, alpha=1, label='Training Data')
plt.hist(b, bins, alpha=1, label='Validation Data')
plt.hist(c, bins, alpha=0.6, label='Test Data')
plt.legend(loc='upper center')

plt.show()

In [ ]:
# rotation

degrees = 10
degrees_positive = 10
degrees_negative = 350
X_train_rotated_positive = []
X_train_rotated_negative = []

for i in range(len(X_train)):
    rotated_image = scipy.ndimage.rotate(X_train[i], degrees_positive)
    X_train_rotated_positive.append(rotated_image)

for i in range(len(X_train)):
    rotated_image = scipy.ndimage.rotate(X_train[i], degrees_negative)
    X_train_rotated_negative.append(rotated_image)

for i in range(len(X_train_rotated_positive)):
    X_train_rotated_positive[i] = X_train_rotated_positive[i][2:34,2:34]

for i in range(len(X_train_rotated_negative)):
    X_train_rotated_negative[i] = X_train_rotated_negative[i][2:34,2:34]

X_train = list(X_train)

for i in range(len(X_train_rotated_positive)):
    X_train.append(X_train_rotated_positive[i])
    X_train.append(X_train_rotated_negative[i])

y_train = list(y_train)

for i in range(len(y_train)):
    y_train.append(y_train[i])
    y_train.append(y_train[i])

X_train = np.array(X_train)
new_n_train = len(X_train)

y_train = np.array(y_train)
new_n_train_y = len(y_train)

print("New number of ground truth labels = ", new_n_train_y)
print("New number of training samples after data augmentation = ", new_n_train)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3)
ax1.imshow(X_train[2018])
ax1.set_title('Original')
ax1.axis('ON')
ax2.imshow(X_train_rotated_positive[2018])
ax2.set_title('Rotated +10°')
ax2.axis('ON')
ax3.imshow(X_train_rotated_negative[2018])
ax3.set_title('Rotated -10°')
ax3.axis('ON')
plt.savefig('./examples/Figure_original_rotated.jpg', dpi=300)
plt.show()

In [ ]:
X_train, y_train = shuffle(X_train, y_train)

print("shuffle complete")

In [ ]:
def normalize(image):
    normal = (image - 127.5)/255
    return np.reshape(normal, (32,32,1))

In [ ]:
def grayscale(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return np.reshape(gray, (32,32,1))

In [ ]:
def equalize(image):
    equal = cv2.equalizeHist(image)
    return np.reshape(equal, (32,32,1))

In [ ]:
X_train_normalized = []

for image in X_train:
    gray_image = grayscale(image)
    equal_image = equalize(gray_image)
    normalized_image = normalize(equal_image)
    X_train_normalized.append(normalized_image)

In [ ]:
X_valid_normalized = []

for image in X_valid:
    gray_image = grayscale(image)
    equal_image = equalize(gray_image)
    normalized_image = normalize(equal_image)
    X_valid_normalized.append(normalized_image)

In [ ]:
X_test_normalized = []

for image in X_test:
    gray_image = grayscale(image)
    equal_image = equalize(gray_image)
    normalized_image = normalize(equal_image)
    X_test_normalized.append(normalized_image)

In [ ]:
print('completed')